# Descarga de datos de Fallecimientos desde Sanidad

## Objetivo

Vamos a hacer una lectura de los datos publicados en los informes diarios de datos de Covid19 publicados por Sanidad. Este es un [ejemplo](https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_265_COVID-19.pdf) publicado del 4 de Diciembre 2020.
  
Manuel H. Arias  
[@walyt](https://twitter.com/walyt)  

[#escovid19data](https://github.com/montera34/escovid19data)

versión definitiva documentada, para ser publicar en el repo

[@walyt](https://twitter.com/walyt)

## Código

Tenemos un montón de librerias con las que vamos a trabajar, no he tenido ningún problema en instalar aquellas no disponibles en el entorno Anaconda con el que trabajo por medio de `pip install libreria` realizado desde un terminal abierto desde el entorno `env`.

In [33]:
import os.path as pth
import datetime as dt
import time
from glob import glob
import re
import pandas as pd
import numpy as np

import requests
from shutil import copyfile

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib import cm
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib.dates import (YEARLY, MONTHLY, DateFormatter, WeekdayLocator, MonthLocator,DayLocator,
                              rrulewrapper, RRuleLocator, drange)
import seaborn as sns
import matplotlib.colors as colors

import numpy as np
from datetime import datetime,timedelta
import seaborn as sns
%matplotlib inline

from io import StringIO
import fitz

Preparamos las expresiones regulares que nos ayudarán en la interpretación de la información que sacamos de los pdf.

Definimos variables que nos ayuden en la gestión de los nombres de los ficheros.

In [34]:
fuente_ficheros_pdf_sanidad = '/Users/mharias/documents/proyectos/covid/sanidad_fallecidos/pdfs_sanidad/'
dir_csv = '/Users/mharias/documents/proyectos/covid/sanidad_fallecidos/csv/'
URL_reg='https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_{:02d}_COVID-19.pdf'

### Función para descargar un fichero pdf, copiada del script de [@alfonsotwr](https://github.com/alfonsotwr/snippets/tree/master/covidia-cam)

In [35]:
def descarga(url,num):
    print('Descargando:', url)
    fn=fuente_ficheros_pdf_sanidad+str(num)+'.pdf'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    with requests.Session() as s:
        r = s.get(url, headers=headers)
    if r.status_code == requests.codes.ok:
        with open(fn, 'wb') as fp:
                fp.write(r.content)
    else:
        print ('Error con el ',num)
    return 

### Descarga de un rango o de un solo pdf

Descargamos el rango completo en el caso de que sea la primera vez. Arrancamos con el 77, pues no pude descifrar el formato de los pdf anteriores.

In [34]:
for i in range(303,312):
    descarga(URL_reg.format(i),i)

Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_303_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_304_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_305_COVID-19.pdf


KeyboardInterrupt: 

### Descarga de un solo fichero

Como ejemplo 266 corresponde al Viernes 4 Diciembre 2020, grabamos el fichero en el directorio local con el nº de orden del documento.

In [36]:
descarga(URL_reg.format(329),329)

Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_329_COVID-19.pdf


# Nuevo procedimiento tras los problemas de semana 12/02

In [37]:
comunidades=['Andalucía','Aragón','Asturias','Baleares','Canarias','Cantabria','Castilla La Mancha',
             'Castilla y León','Cataluña','Ceuta','C. Valenciana','Extremadura','Galicia',
             'Madrid','Melilla','Murcia','Navarra','País Vasco','La Rioja']

In [38]:
indice=['Andalucia','Aragon','Asturias','Baleares','Canarias','Cantabria','Castilla La Mancha',
            'Castilla y Leon','Cataluña','Ceuta','C.Valenciana','Extremadura','Galicia','Madrid','Melilla','Murcia',
             'Navarra','Pais Vasco','La Rioja']
datos=pd.DataFrame()

In [39]:
claves={}
indice=76
rango= np.arange(datetime(2020,4,16),datetime.today(),timedelta(days=1)).astype(datetime)
for day in rango:
    if not (((day>datetime(2020,6,30))&(day.weekday() in [5,6])) |\
            (day==datetime(2020,12,6)) | (day==datetime(2020,12,8)) | (day==datetime(2020,12,25)) | \
           (day==datetime(2021,1,1)) | (day==datetime(2021,1,6))):
        #print (day,i,)
        indice+=1
        claves[indice]=day.strftime('%-d/%-m/%Y')

In [40]:
claves

{77: '16/4/2020',
 78: '17/4/2020',
 79: '18/4/2020',
 80: '19/4/2020',
 81: '20/4/2020',
 82: '21/4/2020',
 83: '22/4/2020',
 84: '23/4/2020',
 85: '24/4/2020',
 86: '25/4/2020',
 87: '26/4/2020',
 88: '27/4/2020',
 89: '28/4/2020',
 90: '29/4/2020',
 91: '30/4/2020',
 92: '1/5/2020',
 93: '2/5/2020',
 94: '3/5/2020',
 95: '4/5/2020',
 96: '5/5/2020',
 97: '6/5/2020',
 98: '7/5/2020',
 99: '8/5/2020',
 100: '9/5/2020',
 101: '10/5/2020',
 102: '11/5/2020',
 103: '12/5/2020',
 104: '13/5/2020',
 105: '14/5/2020',
 106: '15/5/2020',
 107: '16/5/2020',
 108: '17/5/2020',
 109: '18/5/2020',
 110: '19/5/2020',
 111: '20/5/2020',
 112: '21/5/2020',
 113: '22/5/2020',
 114: '23/5/2020',
 115: '24/5/2020',
 116: '25/5/2020',
 117: '26/5/2020',
 118: '27/5/2020',
 119: '28/5/2020',
 120: '29/5/2020',
 121: '30/5/2020',
 122: '31/5/2020',
 123: '1/6/2020',
 124: '2/6/2020',
 125: '3/6/2020',
 126: '4/6/2020',
 127: '5/6/2020',
 128: '6/6/2020',
 129: '7/6/2020',
 130: '8/6/2020',
 131: '9/6/202

In [41]:
rango = np.concatenate([np.arange(77,137),np.arange(138,234)])
opt='text'
for i in rango: #original 100 a 234
    fn=fuente_ficheros_pdf_sanidad+str(i)+'.pdf'
    doc = fitz.open(fn)
    numero_pagina=1
    page = doc.load_page(numero_pagina)
    text = page.get_text(opt)
    for comunidad in comunidades:
        regexp = r'{}[¥\*]* '.format(comunidad)+'(((\n\d*[,.]?\d+)[¥* ]* |\n[ ]*|\n\*+ |\n- ){6})'
        pattern = re.compile(regexp)
        cadena = re.search(pattern,text).group(1).replace('¥','').split('\n')[1:]
        
        cadena = [i.rstrip().lstrip().replace('.','') if i not in ['',' '] else 0 for i in cadena]
        c1,c2 = cadena[4:6]
        datos = datos.append({'fecha':claves[i],'ccaa':comunidad,'fallecidos':c1},ignore_index=True)
        #print (i,'->',comunidad,c1,c2)

In [42]:
rango = np.concatenate([np.arange(235,266),np.arange(267,330)])

for i in rango:
    fn=fuente_ficheros_pdf_sanidad+str(i)+'.pdf'
    doc = fitz.open(fn)
    numero_pagina=3 if i==266 else 5 if i==282 else 4
    page = doc.load_page(numero_pagina)
    text = page.get_text(opt)
    #print (text)
    for comunidad in comunidades:
        regexp = r'{} '.format(comunidad)+'((\n\d*[,.]?\d+¥? ){2})'
        pattern = re.compile(regexp)
        c1,c2 = re.search(pattern,text).group(1).replace('\n','').replace('.','').split()
        #print (i,'->',comunidad,c1,c2)
        datos = datos.append({'fecha':claves[i],'ccaa':comunidad,'fallecidos':c1},ignore_index=True)

In [43]:
datos

,ccaa,fallecidos,fecha
0,Andalucía,912,16/4/2020
1,Aragón,543,16/4/2020
2,Asturias,168,16/4/2020
3,Baleares,131,16/4/2020
4,Canarias,107,16/4/2020
...,...,...,...
4745,Melilla,74,10/3/2021
4746,Murcia,1532,10/3/2021
4747,Navarra,1106,10/3/2021
4748,País Vasco,3817,10/3/2021


In [44]:
datos.shape

(4750, 3)

In [45]:
datos.to_csv(dir_csv+'datos_fallecidos_sanidad.csv',index=False,sep=',')